In [1]:
import os
import pandas as pd
import numpy as np
from sim.core.cascade import RepoCascade
from sim.core.inputs import load_inputs, load_k_covid
from sim.zaf.dhiv import ModelHIV, get_intv

In [2]:
iso = 'ZAF'
in_folder = f'results/D_{iso}'
out_folder = f'results/I_{iso}'

In [3]:
os.makedirs(out_folder, exist_ok=True)

In [4]:
pars_cs = pd.read_csv(f'data/pars_{iso}.csv')
repo_cs = RepoCascade(pars_cs, 2010)
k_covid = load_k_covid(f'{in_folder}/k_covid.csv')

inp = load_inputs(f'data/pars/{iso}')
model = ModelHIV(inp)

In [5]:
particles = pd.read_csv(f'{in_folder}/Post.csv')
particles = [dict(row) for _, row in particles.iterrows()]

particles = particles

In [6]:
baseline = list()
for i, p in enumerate(particles):
    p['cas'] = repo_cs.sample()
    p['k_covid'] = k_covid
    ys, ms, _ = model.simulate_to_fit(p, t_eval=np.linspace(2015, 2022, 8))
    baseline.append((p, ys, ms.assign(Key = i, Scenario = 'Baseline')))

In [7]:
rd0 = 2
scs = {
    'ACF2_b': lambda p: get_intv(p, rd_csi=rd0, rd_recsi=rd0),
    'Dx90_ACF2_b': lambda p: get_intv(p, pdx0=0.9, pdx1=0.9, rd_csi=rd0, rd_recsi=rd0, r_asym_acf=0.1)
}

rd1 = 2 / 0.45 * 0.8
scs.update({
    'ACF2_cxr': lambda p: get_intv(p, rd_csi=rd1, rd_recsi=rd1),
    'Dx90_ACF2_cxr': lambda p: get_intv(p, pdx0=0.9, pdx1=0.9, rd_csi=rd1, rd_recsi=rd1, r_asym_acf=0.1)
})

rd2 = 2 / 0.45 * 0.95
scs.update({
    'ACF2_xpert': lambda p: get_intv(p, rd_csi=rd2, rd_recsi=rd2),
    'Dx90_ACF2_xpert': lambda p: get_intv(p, pdx0=0.9, pdx1=0.9, rd_csi=rd2, rd_recsi=rd2, r_asym_acf=0.1)
})

In [8]:
intvs = list()

for i, (p, ys, _) in enumerate(baseline):
    _, ms0, _ = model.simulate_intv(p, ys)
    intvs.append(ms0.assign(Key = i, Scenario = 'Baseline'))
    
    for k, sc in scs.items():
        intv = sc(p)
        _, ms1, _ = model.simulate_intv(p, ys, intv)
        intvs.append(ms1.assign(Key = i, Scenario = k))

intvs = pd.concat(intvs)

In [9]:
intvs.to_csv(f'{out_folder}/RunSens.csv')

In [10]:
len(intvs)

4900